<a href="https://colab.research.google.com/github/aleshkovskijjaaa/--PY1/blob/main/Lab_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Лабораторная работа**: 1

**Автор**: Алешковский Александр Анатольевич

**Группа**: 5140901/31502

**Глава 1**: Расширенное введение в совместное и параллельное программирование

Допустим, что у нас имеется некая простая функция, которая проверяет является ли некое неотрицательное число простым следующим образом:

In [ ]:
from math import sqrt

def is_prime(x):
    if x < 2:
      return False

    if x == 2:
      return True

    if x % 2 == 0:
      return False

    limit = int(sqrt(x)) + 1
    for i in range(3, limit, 2):
      if x % i == 0:
        return False

    return True

Кроме того, допустим что у нас имеется некий перечень достаточно больших целых (10^13 до 10^13 + 500)

, и мы желаем проверить являются ли каждое из них простым путём применения нашей предыдущей функции:

In [ ]:
input = [i for i in range(10 ** 13, 10 ** 13 + 500)]

Некий последовательный подход будет простой передачей одного числа за другим в нашу функцию is_prime() следующим образом:

In [ ]:
from timeit import default_timer as timer

# sequential
start = timer()
result = []
for i in input:
    if is_prime(i):
        result.append(i)
print('Result 1:', result)
print('Took: %.2f seconds.' % (timer() - start))

Result 1: [10000000000037, 10000000000051, 10000000000099, 10000000000129, 10000000000183, 10000000000259, 10000000000267, 10000000000273, 10000000000279, 10000000000283, 10000000000313, 10000000000343, 10000000000391, 10000000000411, 10000000000433, 10000000000453]
Took: 5.68 seconds.


Эта программа потребовала почти 5 секунд на обработку всех чисел

Теперь  посмотрим, сможет ли на самом деле параллельность помочь нам улучшить нашу программу. Наша функция is_prime() содержит много тяжёлых вычислений и, таким образом, является хорошим претендентом на параллельное программирование.

In [ ]:
import concurrent.futures
start = timer()
result = []
with concurrent.futures.ProcessPoolExecutor(max_workers=20) as executor:
    futures = [executor.submit(is_prime, i) for i in input]

    for i, future in enumerate(concurrent.futures.as_completed(futures)):
        if future.result():
            result.append(input[i])

print('Result 2:', result)
print('Took: %.2f seconds.' % (timer() - start))

Result 2: [10000000000484, 10000000000485, 10000000000486, 10000000000487, 10000000000488, 10000000000489, 10000000000490, 10000000000491, 10000000000492, 10000000000493, 10000000000494, 10000000000495, 10000000000496, 10000000000497, 10000000000498, 10000000000499]
Took: 4.63 seconds.


Основное ключевое отличие между совместным и параллельным программированием состоит в том, что в параллельной программе существует некое число потоков обработки (в основном ЦПУ и ядер), работающих независимо по одному каждый, причём могут иметься различные потоки обработки (в основном потоки, threads), выполняющие доступ и использующие некий разделяемый ресурс в одно и то же время в программах совместной работы.

Чтобы продемонстрировать само понятие запуска множества потоков в одном и том же процессе, давайте рассмотрим некий быстрый пример на Python.

In [ ]:
import threading
import time


class MyThread(threading.Thread):
    def __init__(self, name, delay):
        threading.Thread.__init__(self)
        self.name = name
        self.delay = delay

    def run(self):
        print('Starting thread %s.' % self.name)
        thread_count_down(self.name, self.delay)
        print('Finished thread %s.' % self.name)

def thread_count_down(name, delay):
    counter = 5

    while counter:
        time.sleep(delay)
        print('Thread %s counting down: %i...' % (name, counter))
        counter -= 1

В этом файле мы применяем соответствующий модуль threading мз Python в качестве необходимой основы для своего класса MyThread. Кадлый объект из этого класса имеет некое name и параметр delay. Имеющаяся функция run(), которая вызывается как только некий новый поток инициализирован и запущен, печатает какое- то стартовое сообщение и, в свою очередь, вызывает соответствующую функцию thread_count_down(). Данная функция ведёт обратный отсчёт с 5 до 0, а между итерациями засыпает на несколько секунд, которые определены определяемым параметром задержки.

Основной момент в данном примере состоит в том чтобы показать имеющуюся природу совместной обработки через одновременный запуск более одного объектов из нашего класса MyThread. Мы знаем, что как только каждый поток запускается, также стартует и обратный отсчёт на основе времени. В традиционной последовательной программе отдельный обратный отсчёт будет исполняться обособленно, по- порядку (то есть, какой- то новый обратный отсчёт не начнётся, пока не завершится текущий). Как вы обнаружите, все отдельные обратные отсчёты для обособленных потоков исполняются совместно.

In [ ]:
thread1 = MyThread('A', 0.5)
thread2 = MyThread('B', 0.5)

thread1.start()
thread2.start()

thread1.join()
thread2.join()


print('Finished.')

Starting thread A.
Starting thread B.
Thread A counting down: 5...
Thread B counting down: 5...
Thread A counting down: 4...
Thread B counting down: 4...
Thread A counting down: 3...
Thread B counting down: 3...
Thread A counting down: 2...
Thread B counting down: 2...
Thread A counting down: 1...
Finished thread A.
Thread B counting down: 1...
Finished thread B.
Finished.


 Полученный вывод сообщает нам, что наши два обратных отсчёта для имеющихся потоков исполнялись совместно; вместо того чтобы завершить наш самый первый поток обратного отсчёта и затем запустить второй имеющийся поток обратного отсчёта, наша программа исполнила эти два обратных отсчёта почти в одно и то же время. Без включения каких- то накладных расходов и различных объявлений, такая методика организации потоков позволяет почти удвоить улучшение скорости для нашей предыдущей программы.

 У нас есть возможность спроектировать некую структуру, в которой наш пул потоков не будет сохранять какую бы то ни было информацию относящуюся к задачам, которые ему следует исполнять, а вместо этого сами задачи хранятся в некоторой очереди (иными словами, создают очередь задач), и соответствующие элементы из этой очереди будут питать персональных участников имеющегося пула потоков. По мере того как выбранная задача завершается неким участником нашего пула потоков, если присутствующая очередь задач всё ещё содержит подлежащие обработке элементы, тогда следующий элемент из этой очереди будет отправлен в тот поток, который только что освободился.

**В этом примере мы будем рассматривать задачу вывода всех положительных множителей некоторых элементов в заданном списке положительных целых.**

In [ ]:
import queue
import threading
import time


class MyThread(threading.Thread):
    def __init__(self, name):
        threading.Thread.__init__(self)
        self.name = name

    def run(self):
        print('Starting thread %s.' % self.name)
        process_queue()
        print('Exiting thread %s.' % self.name)

def process_queue():
    while True:
        try:
            x = my_queue.get(block=False)
        except queue.Empty:
            return
        else:
            print_factors(x)

        time.sleep(1)

def print_factors(x):
    result_string = 'Positive factors of %i are: ' % x
    for i in range(1, x + 1):
        if x % i == 0:
            result_string += str(i) + ' '
    result_string += '\n' + '_' * 20

    print(result_string)


# setting up variables
input_ = [1, 10, 4, 3]

# filling the queue
my_queue = queue.Queue()
for x in input_:
    my_queue.put(x)


# initializing and starting 3 threads
thread1 = MyThread('A')
thread2 = MyThread('B')
thread3 = MyThread('C')

thread1.start()
thread2.start()
thread3.start()

# joining all 3 threads
thread1.join()
thread2.join()
thread3.join()

print('Done.')

Starting thread A.
Positive factors of 1 are: 1 
____________________
Starting thread B.Starting thread C.
Positive factors of 10 are: 1 2 5 10 
____________________

Positive factors of 4 are: 1 2 4 
____________________
Positive factors of 3 are: 1 3 
____________________
Exiting thread C.
Exiting thread B.
Exiting thread A.
Done.


**Глава 2 - Одновременные web - запросы**

С точки зрения очереди в многопоточности каждый поток извлекает элементы из очереди, обрабатывает их и завершается, когда очередь опустеет. Это позволяет эффективно распределить работу между несколькими потоками и обрабатывать элементы из очереди параллельно.






In [ ]:
import requests

url = 'http://www.google.com'

res = requests.get(url)

print(res.status_code)
print(res.headers)

with open('google.html', 'w') as f:
    f.write(res.text)

print('Done.')

200
{'Date': 'Thu, 18 Apr 2024 11:27:59 GMT', 'Expires': '-1', 'Cache-Control': 'private, max-age=0', 'Content-Type': 'text/html; charset=ISO-8859-1', 'Content-Security-Policy-Report-Only': "object-src 'none';base-uri 'self';script-src 'nonce-MaHkXg-GlilvEfqCJTRn7Q' 'strict-dynamic' 'report-sample' 'unsafe-eval' 'unsafe-inline' https: http:;report-uri https://csp.withgoogle.com/csp/gws/other-hp", 'P3P': 'CP="This is not a P3P policy! See g.co/p3phelp for more info."', 'Content-Encoding': 'gzip', 'Server': 'gws', 'Content-Length': '8586', 'X-XSS-Protection': '0', 'X-Frame-Options': 'SAMEORIGIN', 'Set-Cookie': '1P_JAR=2024-04-18-11; expires=Sat, 18-May-2024 11:27:59 GMT; path=/; domain=.google.com; Secure, AEC=AQTF6HwcR-Z1_KH09yHKe_LHYHalbuCF-4O14W_CGO_fI6HYq9J2Bq6thXA; expires=Tue, 15-Oct-2024 11:27:59 GMT; path=/; domain=.google.com; Secure; HttpOnly; SameSite=lax, NID=513=fbbvYA7Frvbmj_u7adyOQjI3Q3k4cSsH2AcucmUgzGrJULQoG3_9ec4bfX0sqtJzl9qs9pFg4uNJJzVRHqe8HvCCrmZmZEvrMydd5fGTDbJ9Q-L62F

В этом примере мы применяем модуль requests для выгрузки кода обозначенной веб страницы, www.google.com. Соответствующий метод requests.get() отправляет некий метод запроса GET в url, а мы сохраняем полученный отклик в своей переменной res. После проверки значения состояния и заголовков отклика через их вывод на печать, мы создаём некий файл с названием google.html и записываем тот код HTML, который хранится в самом тексте отклика в данный файл.

In [ ]:
import requests

def ping(url):
    res = requests.get(url)
    print(f'{url}: {res.text}')

urls = [
    'http://httpstat.us/200',
    'http://httpstat.us/400',
    'http://httpstat.us/404',
    'http://httpstat.us/408',
    'http://httpstat.us/500',
    'http://httpstat.us/524'
]

for url in urls:
    ping(url)

print('Done.')

http://httpstat.us/200: 200 OK
http://httpstat.us/400: 400 Bad Request
http://httpstat.us/404: 404 Not Found
http://httpstat.us/408: 408 Request Timeout
http://httpstat.us/500: 500 Internal Server Error
http://httpstat.us/524: 524 A Timeout Occurred
Done.


В этой программе наша функция ping() получает некий URL и пытается выполнить запрос GETк этому сайту. Затем она выведет на печать полученное содержимое соответствующего отклика, возвращаемого вызывавшимся сервером. В нашей основной программе у нас имеется перечень различных кодов состояния, которые мы уже упоминали ранее, причём все они будут подвергнуты обходу и будут вызваны этой функцией ping().

**Одновременные веб запросы**

В контексте совместного программирования мы можем обнаружить, что определённый процесс выполнения какого- то запроса к веб серверу и получению соответствующего возвращаемого отклика независим от той же самой процедуры для какого- то другого веб сервера. Это говорит о том, что мы можем применить совместную обработку и параллелизм в своём приложении ping для ускорения нашего исполнения.

In [ ]:
import threading
import requests
import time

def ping(url):
    res = requests.get(url)
    print(f'{url}: {res.text}')

urls = [
    'http://httpstat.us/200',
    'http://httpstat.us/400',
    'http://httpstat.us/404',
    'http://httpstat.us/408',
    'http://httpstat.us/500',
    'http://httpstat.us/524'
]

start = time.time()
for url in urls:
    ping(url)
print(f'Sequential: {time.time() - start : .2f} seconds')

print()

start = time.time()
threads = []
for url in urls:
    thread = threading.Thread(target=ping, args=(url,))
    threads.append(thread)
    thread.start()
for thread in threads:
    thread.join()

print(f'Threading: {time.time() - start : .2f} seconds')

http://httpstat.us/200: 200 OK
http://httpstat.us/400: 400 Bad Request
http://httpstat.us/404: 404 Not Found
http://httpstat.us/408: 408 Request Timeout
http://httpstat.us/500: 500 Internal Server Error
http://httpstat.us/524: 524 A Timeout Occurred
Sequential:  0.86 seconds

http://httpstat.us/404: 404 Not Found
http://httpstat.us/200: 200 OK
http://httpstat.us/408: 408 Request Timeout
http://httpstat.us/500: 500 Internal Server Error
http://httpstat.us/524: 524 A Timeout Occurred
http://httpstat.us/400: 400 Bad Request
Threading:  0.13 seconds
